In [3]:
from RCN import *
import numpy as np

In [2]:
# calibrated parameters
r = -0.007801729853464256
delta = 0.02720239796079839
U = 1.054699999999994
D = 0.9440001342869291

### 1)
#### i)

In [5]:
payment_dates = np.arange(1,13)
period_length = 1/12
coupon_rate = 0.10
initial_price = 11118
exercise_price = 1

In [6]:
rcn1 = RCN_binomial(r, period_length, initial_price, delta, U, D,
                    payment_dates, coupon_rate, exercise_price, Simple=True, Callable=False, barrier_level=None)

In [10]:
rcn1.price_RCN()

1.0124083511877382

#### ii)

In [12]:
barrier_level = 0.8
rcn2 = RCN_binomial(r, period_length, initial_price, delta, U, D,
                    payment_dates, coupon_rate, exercise_price, Simple=False, Callable=False, barrier_level=barrier_level)

In [16]:
rcn2.set_barrier()
rcn2.price_RCN()

1.0544990135271406